In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger


In [2]:
import re
import fitz
import pandas as pd

import sys
sys.path.append('E:\Code\chat\gdpr')

import os
path = 'E:/Code/chat/gdpr/pdf/guidlines'
file_name = '20180419_art29_wp_position_paper_art_30_publish_AF01E86A-01CE-DBDD-6368D0296A2413B0_51422.pdf'
paper_name = 'WORKING PARTY 29 POSITION PAPER on the derogations from the obligation to maintain records of processing activities pursuant to Article 30(5) GDPR'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

In [17]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
document_text = output_doc_as_text(doc, start_page = 0, end_page = 0, header_size=80, footer_size=40, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

In [18]:
# remove footnote on page 1
document_text = document_text.replace("This Working Party was set up under Article 29 of Directive 95/46/EC. It is an independent European advisory body on data protection and privacy. Its tasks are described in Article 30 of Directive 95/46/EC and Article 15 of Directive 2002/58/EC. The secretariat is provided by Directorate C (Fundamental Rights and Union Citizenship) of the European Commission, Directorate General Justice, B-1049 Brussels, Belgium, Office No MO-59 02/013. Website: http://ec europa eu/newsroom/article29/news cfm?item type=1358&tpa id=6936 WORKING PARTY 29 POSITION PAPER on the derogations from the obligation to maintain records of processing activities pursuant to Article 30(5) GDPR", "")
# add paragraphs
document_text = document_text.replace("However, such organisations need only maintain records of processing activities for the types of processing mentioned by Article 30(5). ", "\nHowever, such organisations need only maintain records of processing activities for the types of processing mentioned by Article 30(5).\n")
document_text = document_text.replace("personal data relating to criminal convictions and offences. ", "personal data relating to criminal convictions and offences.\n")
document_text = document_text.replace("both key components of the principle of accountability contained in the GDPR. ", "both key components of the principle of accountability contained in the GDPR.\n")
document_text = document_text.replace("SMEs to keep records of processing activities not covered by the derogation in Article 30(5).", "SMEs to keep records of processing activities not covered by the derogation in Article 30(5).\n\n")

print(document_text)


The Working Party 29 has examined the obligation, under Article 30 of the GDPR, for controllers and processors to maintain a record of processing activities. This paper sets out the WP29's position on the derogation from this obligation. Recital 13 of the GDPR says:
'To take account of the specific situation of micro, small and medium-sized enterprises, this Regulation includes a derogation for organisations with fewer than 250 employees with regard to record-keeping'.
Article 30(5) gives effect to Recital 13. It says that the obligation to keep a record of processing activities does not apply 'to an enterprise or an organisation employing fewer than 250 persons unless the processing it carries out is likely to result in a risk to the rights and freedoms of data subjects, the processing is not occasional, or the processing includes special categories of data as referred to in Article 9(1) or personal data relating to criminal convictions and offences referred to in Article 10.' Some c

In [27]:
from regulations_rag.embeddings import num_tokens_from_string
print(f"The document consists of {num_tokens_from_string(document_text)} tokens")

data = [["", False, document_text]]
df=pd.DataFrame(data, columns = ["section_reference", "heading", "text"])
df.to_csv("../inputs/documents/article_30_5.csv", sep = "|", encoding = "utf-8", index = False)

The document consists of 853 tokens


In [19]:
import importlib
import regulations_rag.reference_checker
importlib.reload(regulations_rag.reference_checker)
from regulations_rag.reference_checker import ReferenceChecker
class EmptyReferenceChecker(ReferenceChecker):
    def __init__(self):
        exclusion_list = []
        index_patterns = [r""]    
        text_pattern = ""

        super().__init__(regex_list_of_indices = index_patterns, text_version = text_pattern, exclusion_list=exclusion_list)

    def is_valid(self, reference):
        if reference == "":
            return True
        return False

    def extract_valid_reference(self, input_string):
        return ""
    
    def split_reference(self, reference):
        return ""

    def get_parent_reference(self, input_string):
        return ""

    def get_current_and_parent_references(self, reference):
        return [""]

    def is_reference_or_parents_in_list(self, reference, list_of_references):
        super().is_reference_or_parents_in_list(reference, list_of_references) 

    def _extract_reference_from_string(self, s):
        return ""

no_index = EmptyReferenceChecker()

In [20]:
unindex.is_valid(r"")

False

In [32]:
a_30_5 = Article_30_5()

In [34]:
print(a_30_5.get_text(""))


The Working Party 29 has examined the obligation, under Article 30 of the GDPR, for controllers and processors to maintain a record of processing activities. This paper sets out the WP29's position on the derogation from this obligation. Recital 13 of the GDPR says:
'To take account of the specific situation of micro, small and medium-sized enterprises, this Regulation includes a derogation for organisations with fewer than 250 employees with regard to record-keeping'.
Article 30(5) gives effect to Recital 13. It says that the obligation to keep a record of processing activities does not apply 'to an enterprise or an organisation employing fewer than 250 persons unless the processing it carries out is likely to result in a risk to the rights and freedoms of data subjects, the processing is not occasional, or the processing includes special categories of data as referred to in Article 9(1) or personal data relating to criminal convictions and offences referred to in Article 10.' Some c

In [45]:
import os
from regulations_rag.standard_regulation_index import load_parquet_data
key = os.getenv('encryption_key_gdpr')
index_df = load_parquet_data("../inputs/index_gdpr.parquet", key)

In [47]:
from openai import OpenAI
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)

import src.summarise_and_question
importlib.reload(src.summarise_and_question)
from src.summarise_and_question import get_summary_and_questions_for


In [49]:
#model = "gpt-3.5-turbo"
#model="gpt-4"
model = "gpt-4-0125-preview"
model = "gpt-4-turbo-2024-04-09"

reg_text = a_30_5.get_text("")
print("##############")
print(reg_text)
print("##############")

model_summary, model_questions = get_summary_and_questions_for(openai_client = openai_client, text = reg_text, model = model)

#format output
article = sectioned_df.loc[index]['article']
print(f'df_summary.loc[index, "article"] = "{article}"')
print(f'df_summary.loc[index, "text"] = "{model_summary}"')
print()
print(f'df_questions.loc[index, "article"] = "{article}"')
print(f'df_questions.loc[index, "text"] = "{model_questions}"')

##############

The Working Party 29 has examined the obligation, under Article 30 of the GDPR, for controllers and processors to maintain a record of processing activities. This paper sets out the WP29's position on the derogation from this obligation. Recital 13 of the GDPR says:
'To take account of the specific situation of micro, small and medium-sized enterprises, this Regulation includes a derogation for organisations with fewer than 250 employees with regard to record-keeping'.
Article 30(5) gives effect to Recital 13. It says that the obligation to keep a record of processing activities does not apply 'to an enterprise or an organisation employing fewer than 250 persons unless the processing it carries out is likely to result in a risk to the rights and freedoms of data subjects, the processing is not occasional, or the processing includes special categories of data as referred to in Article 9(1) or personal data relating to criminal convictions and offences referred to in Arti

NameError: name 'sectioned_df' is not defined

In [50]:
model_summary

"If you have fewer than 250 employees, you usually don't need to keep a record of processing activities. However, this exemption does not apply if any of the following conditions are met:\n\n1. The processing is likely to result in a risk to the rights and freedoms of individuals.\n2. The processing is not occasional, meaning it is carried out regularly or is part of the regular business.\n3. The processing involves special categories of data as outlined in Article 9(1), or personal data relating to criminal convictions and offences as referred to in Article 10.\n\nIf any one of these conditions is met, you must maintain a record of those processing activities, even if other activities can be exempt under this rule. For instance, regular processing of employee data must be recorded, but occasional processing that poses no risk and does not involve special data categories might not need to be documented.\n\nMaintaining a record of processing activities aids in assessing the implications

In [51]:
model_questions

'What are the record-keeping exceptions for organisations with fewer than 250 employees?|When must smaller organisations maintain a record of processing activities despite the general exception?|What types of processing activities still require record-keeping even for smaller organisations?|How do the rights and freedoms of individuals affect the record-keeping requirements for small organisations?|Why is maintaining a record of processing activities important?|How can national Supervisory Authorities support small to medium-sized enterprises in managing records of processing activities?|What defines occasional processing in the context of GDPR?|What assistance is available for SMEs to manage their record-keeping obligations?|How does the principle of accountability related to record-keeping in GDPR?'

In [62]:
df_summary = pd.DataFrame([], columns = ["section_reference", "text", "source", "embedding", "document"])
df_questions = pd.DataFrame([], columns = ["section_reference", "text", "source", "embedding", "document"])


In [55]:
index = 0
print(f'df_summary.loc[index, "document"] = "article_30_5"')
print(f'df_summary.loc[index, "text"] = "{model_summary}"')
print()
print(f'df_questions.loc[index, "article"] = "article_30_5"')
print(f'df_questions.loc[index, "text"] = "{model_questions}"')


df_summary.loc[index, "document"] = "article_30_5"
df_summary.loc[index, "text"] = "If you have fewer than 250 employees, you usually don't need to keep a record of processing activities. However, this exemption does not apply if any of the following conditions are met:

1. The processing is likely to result in a risk to the rights and freedoms of individuals.
2. The processing is not occasional, meaning it is carried out regularly or is part of the regular business.
3. The processing involves special categories of data as outlined in Article 9(1), or personal data relating to criminal convictions and offences as referred to in Article 10.

If any one of these conditions is met, you must maintain a record of those processing activities, even if other activities can be exempt under this rule. For instance, regular processing of employee data must be recorded, but occasional processing that poses no risk and does not involve special data categories might not need to be documented.

Maint

In [63]:
df_summary.loc[index, "document"] = "article_30_5"
df_summary.loc[index, "text"] = "If you have fewer than 250 employees, you usually don't need to keep a record of processing activities. However, this exemption does not apply if any of the following conditions are met:\n\
1. The processing is likely to result in a risk to the rights and freedoms of individuals.\n\
2. The processing is not occasional, meaning it is carried out regularly or is part of the regular business.\n\
3. The processing involves special categories of data as outlined in Article 9(1), or personal data relating to criminal convictions and offences as referred to in Article 10.\n\
If any one of these conditions is met, you must maintain a record of those processing activities, even if other activities can be exempt under this rule. For instance, regular processing of employee data must be recorded, but occasional processing that poses no risk and does not involve special data categories might not need to be documented.\n\
Maintaining a record of processing activities aids in assessing the implications of processing on individual rights and helps identify necessary security measures, fulfilling the accountability principle of the GDPR.\n\
Although this might introduce a new administrative task, it is generally not considered onerous for micro, small, and medium-sized enterprises. Supervisory Authorities are encouraged to offer support, such as simplified recording templates, to facilitate compliance for smaller organisations."

df_questions.loc[index, "document"] = "article_30_5"
df_questions.loc[index, "text"] = "What are the record-keeping exceptions for small organisations?|When must smaller organisations maintain a record of processing activities despite the general exception?|What types of processing activities still require record-keeping even for smaller organisations? |Why is maintaining a record of processing activities important? |What defines occasional processing?|What assistance is available for SMEs to manage their record-keeping obligations?|How does the principle of accountability related to record-keeping in GDPR?"


In [64]:
df_questions = df_questions.drop("text", axis=1).join(df_questions["text"].str.split("|", expand=True).stack().reset_index(level=1, drop=True).rename("text"))

In [67]:
df_index = pd.concat([df_summary, df_questions], ignore_index = True)

In [70]:
from regulations_rag.embeddings import get_ada_embedding
model = "text-embedding-3-large"
dimensions = 1024

increment = 10
for i in range(0, len(df_index), increment):
    chunk = df_index.iloc[i:i+increment].copy()
    chunk["embedding"] = chunk["text"].apply(lambda x: get_ada_embedding(openai_client, x, model, dimensions))
    df_index.loc[chunk.index, "embedding"] = chunk["embedding"]
    print(f"Completed {i+increment} lines")


Completed 10 lines


In [78]:
index_df = load_parquet_data("../inputs/index_gdpr.parquet", key)

In [75]:
updated_index = pd.concat([index_df, df_index], ignore_index=True)

In [77]:
save_parquet_data(updated_index, "../inputs/index_gdpr.parquet", key)


In [99]:
index_df = load_parquet_data("../inputs/index_gdpr.parquet", key)

index_df[index_df['text'] == 'Under what conditions could you be exempt from keeping records of your data processing?']


# for index, row in index_df[index_df["section_reference"] == '30'].iterrows():
#     print(f"* {row['text']}")

,section_reference,text,source,embedding,document
147,None,Under what conditions could you be exempt from...,None,"[-0.03439036011695862, -0.0008122794097289443,...",article_30_5


In [94]:
index_df.at[147, "section_reference"] = None
index_df.at[147, "source"] = None
index_df.at[147, "document"] = "article_30_5"



In [96]:
index_df[index_df['text'] == 'Under what conditions could you be exempt from keeping records of your data processing?']

,section_reference,text,source,embedding,document
147,None,Under what conditions could you be exempt from...,None,"[-0.03439036011695862, -0.0008122794097289443,...",article_30_5


In [98]:
save_parquet_data(index_df, "../inputs/index_gdpr.parquet", key)

In [100]:
index_df[index_df['document'] == "article_30_5"]

,section_reference,text,source,embedding,document
147,None,Under what conditions could you be exempt from...,None,"[-0.03439036011695862, -0.0008122794097289443,...",article_30_5
647,None,"If you have fewer than 250 employees, you usua...",None,"[-0.026084575802087784, 0.014087163843214512, ...",article_30_5
648,None,What are the record-keeping exceptions for sma...,None,"[0.007615841459482908, 0.02034611813724041, -0...",article_30_5
649,None,When must smaller organisations maintain a rec...,None,"[0.00046261242823675275, 0.006825227290391922,...",article_30_5
650,None,What types of processing activities still requ...,None,"[0.011152287945151329, 0.021851062774658203, -...",article_30_5
651,None,Why is maintaining a record of processing acti...,None,"[-0.011760201305150986, -0.011568896472454071,...",article_30_5
652,None,What defines occasional processing?,None,"[-0.0750560611486435, 0.04036270081996918, -0....",article_30_5
653,None,What assistance is available for SMEs to manag...,None,"[0.011198482476174831, 0.03537283092737198, -0...",article_30_5
654,None,How does the principle of accountability relat...,None,"[-0.045358121395111084, -0.04111029952764511, ...",article_30_5
